In [7]:
# 完全清空变量状态
%reset -f  # Jupyter中用


Don't know how to reset  #, please run `%reset?` for details
Don't know how to reset  jupyter中用, please run `%reset?` for details


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# 读取数据
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

# 添加标签列用于区分
train["DataSplit"] = "train"
test["DataSplit"] = "test"

# 添加 SalePrice 占位列，保持结构统一
test["SalePrice"] = np.nan

# 合并数据
df_all = pd.concat([train, test], axis=0, ignore_index=True)

# ✅ 确保 DataSplit 存在
assert "DataSplit" in df_all.columns, "❌ df_all中缺失 DataSplit 列！"

# 可视化缺失值比例（可选）
missing = df_all.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
missing_pct = (missing / df_all.shape[0]).sort_values(ascending=False)

plt.figure(figsize=(12,6))
missing_pct.plot(kind='bar')
plt.title("Missing Value Percentage by Feature")
plt.ylabel("Percentage")
plt.tight_layout()
plt.savefig("missing_values.png")
plt.close()

# 删除缺失超过40%的列
to_drop = missing_pct[missing_pct > 0.4].index.tolist()
df_all.drop(columns=to_drop, inplace=True)

# 填充缺失值
for col in df_all.columns:
    if df_all[col].dtype == "object":
        df_all[col] = df_all[col].fillna(df_all[col].mode()[0])
    elif df_all[col].isnull().sum() > 0:
        df_all[col] = df_all[col].fillna(df_all[col].median())

# 特征构造
df_all["TotalSF"] = df_all["TotalBsmtSF"] + df_all["1stFlrSF"] + df_all["2ndFlrSF"]

# 类别变量独热编码
df_all = pd.get_dummies(df_all, drop_first=True)

# 标准化数值特征（除去 SalePrice）
num_cols = df_all.select_dtypes(include=["int64", "float64"]).columns
num_cols = [col for col in num_cols if col != "SalePrice"]
scaler = StandardScaler()
df_all[num_cols] = scaler.fit_transform(df_all[num_cols])

# 拆回训练集和测试集
# 使用 train.index 和 test.index 判断行来源
df_all["is_train"] = df_all.index.isin(train.index)

# 拆分
df_train = df_all[df_all["is_train"] == True].copy()
df_test = df_all[df_all["is_train"] == False].copy()

# 添加 SalePrice 回训练集
df_train["SalePrice"] = train_labels.values

# 删除辅助列
df_train.drop(columns=["is_train"], inplace=True)
df_test.drop(columns=["is_train"], inplace=True)

import os
os.makedirs("processed", exist_ok=True)

# 保存处理好的文件
df_train.to_csv("processed/processed_train.csv", index=False)
df_test.drop(columns=["SalePrice"], errors="ignore").to_csv("processed/processed_test.csv", index=False)


In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# 读取预处理后的数据
df = pd.read_csv("processed/processed_train.csv")

# 分离特征和标签
X = df.drop(columns=["SalePrice"])
y = df["SalePrice"]

# 划分训练集和验证集
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# 初始化模型
model = LinearRegression()

# 训练模型
model.fit(X_train, y_train)

# 预测验证集
y_pred = model.predict(X_valid)

# 评估指标
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
mae = mean_absolute_error(y_valid, y_pred)
r2 = r2_score(y_valid, y_pred)

# 输出评估结果
print(f"RMSE（均方根误差）: {rmse:.2f}")
print(f"MAE（平均绝对误差）: {mae:.2f}")
print(f"R²（判定系数）: {r2:.4f}")


RMSE（均方根误差）: 51988.94
MAE（平均绝对误差）: 20385.86
R²（判定系数）: 0.6476


In [2]:
import pandas as pd
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# 加载数据
df = pd.read_csv("processed/processed_train.csv")
X = df.drop(columns=["SalePrice"])
y = df["SalePrice"]

# 划分训练集和验证集
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# ① Ridge回归（L2正则化）
ridge_model = Ridge(alpha=10)
ridge_model.fit(X_train, y_train)
ridge_preds = ridge_model.predict(X_val)

# ② Lasso回归（L1正则化）
lasso_model = Lasso(alpha=0.001)
lasso_model.fit(X_train, y_train)
lasso_preds = lasso_model.predict(X_val)

# 评估函数
def evaluate(y_true, y_pred, name="Model"):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"【{name}】 RMSE: {rmse:.2f}, MAE: {mae:.2f}, R²: {r2:.4f}")

# 打印结果
evaluate(y_val, ridge_preds, name="Ridge")
evaluate(y_val, lasso_preds, name="Lasso")


【Ridge】 RMSE: 30670.48, MAE: 18587.68, R²: 0.8774
【Lasso】 RMSE: 51949.51, MAE: 20375.84, R²: 0.6482


/opt/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.304e+11, tolerance: 6.967e+08
  model = cd_fast.enet_coordinate_descent(


In [3]:
# 加载处理好的测试集
df_test = pd.read_csv("processed/processed_test.csv")

# 使用 Ridge 模型进行预测
ridge_preds_test = ridge_model.predict(df_test)

# 读取原始 test.csv 获取 PassengerId
df_test_raw = pd.read_csv("data/test.csv")
submission = pd.DataFrame({
    "Id": df_test_raw["Id"],
    "SalePrice": ridge_preds_test
})

# 保存提交文件
submission.to_csv("ridge_submission.csv", index=False)
print("✅ Ridge预测文件已生成：ridge_submission.csv")


✅ Ridge预测文件已生成：ridge_submission.csv
